In [2]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)

Using Device: cuda


In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
train_data = torchvision.datasets.CIFAR10(root='./data', train = True,
                                          transform = transform, 
                                          download = True
                                         )
test_data = torchvision.datasets.CIFAR10(root='./data', train = False,
                                          transform = transform, 
                                          download = True
                                         )
train_loader = torch.utils.data.DataLoader(train_data, 
                                           batch_size = 32,
                                           shuffle = True,
                                           num_workers = 2
                                          )
test_loader = torch.utils.data.DataLoader(test_data, 
                                           batch_size = 32,
                                           shuffle = True,
                                           num_workers = 2
                                          )

100%|███████████████████████████████████████████████████████████████████████████████| 170M/170M [00:50<00:00, 3.35MB/s]


In [7]:
image, label = train_data[0]

In [8]:
class_names = ['plane', 
               'car',
               'bird',
               'cat',
               'deer',
               'dog',
               'frog',
               'horse',
               'ship',
               'truck']

In [9]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward (self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [12]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)
for epoch in range(70):
    print(f'Training epoch {epoch}...')

    running_loss =0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0...
Loss: 2.1972
Training epoch 1...
Loss: 1.7297
Training epoch 2...
Loss: 1.5109
Training epoch 3...
Loss: 1.3987
Training epoch 4...
Loss: 1.3096
Training epoch 5...
Loss: 1.2306
Training epoch 6...
Loss: 1.1642
Training epoch 7...
Loss: 1.1087
Training epoch 8...
Loss: 1.0548
Training epoch 9...
Loss: 1.0048
Training epoch 10...
Loss: 0.9629
Training epoch 11...
Loss: 0.9185
Training epoch 12...
Loss: 0.8824
Training epoch 13...
Loss: 0.8471
Training epoch 14...
Loss: 0.8127
Training epoch 15...
Loss: 0.7835
Training epoch 16...
Loss: 0.7542
Training epoch 17...
Loss: 0.7280
Training epoch 18...
Loss: 0.7012
Training epoch 19...
Loss: 0.6800
Training epoch 20...
Loss: 0.6555
Training epoch 21...
Loss: 0.6336
Training epoch 22...
Loss: 0.6098
Training epoch 23...
Loss: 0.5893
Training epoch 24...
Loss: 0.5655
Training epoch 25...
Loss: 0.5463
Training epoch 26...
Loss: 0.5272
Training epoch 27...
Loss: 0.5078
Training epoch 28...
Loss: 0.4871
Training epoch 29...
Los

In [13]:
net.eval()

NeuralNet(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=600, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [16]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item() 


accuracy = 100 * correct / total
print(f'Accuracy : {accuracy}%')

Accuracy : 67.25%


In [26]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['dog1.jpg', 'horse.jpg', 'plane.jpg', 'horse1.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}' )

Prediction: dog
Prediction: dog
Prediction: plane
Prediction: bird
